In [1]:
import pandas as pd
import numpy as np

In [30]:
from gazprom.mt.pricing.models.swing.fixed_price import FixedPriceSwing
from gazprom.mt.pricing.models.swing.test.swing_market_data import build_swing
from gazprom.mt.pricing.models.exercise import FixedDailyOffset

In [4]:
from gazprom.mt.pricing.environment.environment import Environment
from gazprom.mt.pricing.environment.market_data import CurveData
from gazprom.mt.pricing.models.timeseries.gbmsimulator.volatility_model import VolModelBlended
from gazprom.mt.pricing.models.timeseries.market_data import MonthlyBlendedData

In [31]:
mkt = 'TTF'
ccy = 'EUR'

start_date = pd.datetime(2017, 3, 1)
end_date = pd.datetime(2017, 3, 20)
exercise_dates = pd.date_range(start_date, end_date, freq='D')
tcq_max = 1.0
tcq_min = 0.0
dcq_max = 1.0
dcq_min = 0.0

strike = 25.5

kwargs_payoff = {'strike': strike, 
                 'dcq_max': dcq_max, 'dcq_min': dcq_min, 'early_exercise': FixedDailyOffset(), 
                 'market': mkt, 'ccy': ccy, 'payoff_class': FixedPriceSwing}

In [32]:
payoff = build_swing('call', 'buy', start_date, end_date, tcq_max, tcq_min, **kwargs_payoff)

In [33]:
print(payoff)

FixedPriceSwing [buy, EUR]: market=TTF, type=call,start_date=01-Mar-2017, end_date=20-Mar-2017
# constraints=SwingConstraints(_dcq_min=0.0, _tcq_min=0.0, _tcq_max=1.0, _dcq_max=1.0)
# strike=
2017-03-01    25.5
2017-03-02    25.5
2017-03-03    25.5
2017-03-04    25.5
2017-03-05    25.5
2017-03-06    25.5
2017-03-07    25.5
2017-03-08    25.5
2017-03-09    25.5
2017-03-10    25.5
2017-03-11    25.5
2017-03-12    25.5
2017-03-13    25.5
2017-03-14    25.5
2017-03-15    25.5
2017-03-16    25.5
2017-03-17    25.5
2017-03-18    25.5
2017-03-19    25.5
2017-03-20    25.5
Freq: D, dtype: float64


In [60]:
valuation_date = pd.datetime(2017, 2, 28)

fc = pd.Series(25.5, index=pd.date_range(valuation_date, pd.datetime(2017, 5, 1)))
ir = pd.Series(0.0245, index=exercise_dates)

ctc = pd.date_range(pd.datetime(2017, 1, 1), pd.datetime(2017, 4, 1), freq='MS')
ctc_expiries = pd.Series(ctc - pd.datetools.Day(), index=ctc)
option_expiries = pd.Series(ctc - pd.datetools.MonthEnd(), index=ctc)

volm = pd.Series(0.21, index=ctc)
volb = pd.Series(0.45, index=ctc)

env = Environment(valuation_date, ccy)
env.add_price(ccy, CurveData(ir))
env.add_price(mkt, CurveData(fc))
# env.add_expiry_option(mkt, option_expiries)

mbdata = MonthlyBlendedData(vol_monthly=volm, vol_blended=volb, option_expiry=option_expiries)
env.add_vol(mkt, mbdata, VolModelBlended)

In [61]:
print(env)

Environment (intraday=False):
## _STATE
- currency: IdentityData: data_args={'data_type': <type 'basestring'>}, data:
EUR
- intraday: IdentityData: data_args={'data_type': <type 'bool'>}, data:
False
- pricing_date: IdentityData: data_args={'data_type': <type 'datetime.date'>}, data:
2017-02-28 00:00:00
## TTF
- vol_params: IdentityData: data_args={'data_type': <type 'object'>}, data:
{'offset': 14}
- vol_model: VolModelFunction: vol_kwargs={'builder': 'default', 'calibrator': None}, vol_class=<class 'gazprom.mt.pricing.models.timeseries.gbmsimulator.volatility_model.VolModelBlended'>, market_data=<gazprom.mt.pricing.models.timeseries.market_data.MonthlyBlendedData object at 0x000000000CBC7048>, builder=<gazprom.mt.pricing.environment.data_builders.DefaultDataBuilder object at 0x000000000CBDBBA8>, data_args={}, data:
(KeySpec(key_type=pricing_date, apply_early=False, key=_STATE), KeySpec(key_type=vol_monthly, apply_early=False, key=TTF), KeySpec(key_type=vol_blended, apply_early=False,

In [75]:
simulator_params = {'seed': 12321312, 'simulator': 'SimulatorOneFactor'}
lsmc_params=dict(nsims_fwd=10000, nsims_bwd=10000)

res = payoff.pv_greeks(env, simulator_params=simulator_params, lsmc_params=lsmc_params, debug=True)

In [76]:
res.keys()

['sims',
 'cash_strike',
 'pv',
 'actions_paths',
 'triggers',
 'lsmc_mesh',
 'vega_monthly',
 'fwd_sims',
 'cash',
 'actions',
 'vega_blended',
 'spot_sims',
 'delta',
 'theta',
 'price_model',
 'cashflows']

In [64]:
res['pv']

SV(      1.0555,     0.0075)

In [71]:
res['delta']

SVFrame.value:
                  TTF
2017-03-01  0.000000
2017-03-02  0.000000
2017-03-03  0.000000
2017-03-04  0.000000
2017-03-05  0.000000
2017-03-06  0.000000
2017-03-07  0.000000
2017-03-08  0.111620
2017-03-09  0.000000
2017-03-10  0.000000
2017-03-11  0.110808
2017-03-12  0.000000
2017-03-13  0.000000
2017-03-14  0.000000
2017-03-15  0.000000
2017-03-16  0.108138
2017-03-17  0.000000
2017-03-18  0.000000
2017-03-19  0.000000
2017-03-20  0.218527
SVFrame.err:
                  TTF
2017-03-01  0.000000
2017-03-02  0.000000
2017-03-03  0.000000
2017-03-04  0.000000
2017-03-05  0.000000
2017-03-06  0.000000
2017-03-07  0.000000
2017-03-08  0.105892
2017-03-09  0.000000
2017-03-10  0.000000
2017-03-11  0.105122
2017-03-12  0.000000
2017-03-13  0.000000
2017-03-14  0.000000
2017-03-15  0.000000
2017-03-16  0.102589
2017-03-17  0.000000
2017-03-18  0.000000
2017-03-19  0.000000
2017-03-20  0.138212

In [72]:
res['actions']

2017-03-01    SV(     -0.0000,     0.0000)
2017-03-02    SV(     -0.0000,     0.0000)
2017-03-03    SV(     -0.0000,     0.0000)
2017-03-04    SV(     -0.0000,     0.0000)
2017-03-05    SV(     -0.0000,     0.0000)
2017-03-06    SV(     -0.0000,     0.0000)
2017-03-07    SV(     -0.0000,     0.0000)
2017-03-08    SV(      0.1000,     0.0949)
2017-03-09    SV(     -0.0000,     0.0000)
2017-03-10    SV(     -0.0000,     0.0000)
2017-03-11    SV(      0.1000,     0.0949)
2017-03-12    SV(     -0.0000,     0.0000)
2017-03-13    SV(     -0.0000,     0.0000)
2017-03-14    SV(     -0.0000,     0.0000)
2017-03-15    SV(     -0.0000,     0.0000)
2017-03-16    SV(      0.1000,     0.0949)
2017-03-17    SV(     -0.0000,     0.0000)
2017-03-18    SV(     -0.0000,     0.0000)
2017-03-19    SV(     -0.0000,     0.0000)
2017-03-20    SV(      0.2000,     0.1265)
Freq: D, dtype: object

In [73]:
res['price_model'].vol_model

VolModelBlended(_vol_blended=2017-01-01    0.45
2017-02-01    0.45
2017-03-01    0.45
2017-04-01    0.45
Freq: MS, dtype: float64, _valid_index=DatetimeIndex(['2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01'], dtype='datetime64[ns]', freq='MS'), _blended_offset=14, _monthly_expiries=2017-01-01   2016-12-31
2017-02-01   2017-01-31
2017-03-01   2017-02-28
2017-04-01   2017-03-31
Freq: MS, dtype: datetime64[ns], _vol_monthly=2017-01-01    0.21
2017-02-01    0.21
2017-03-01    0.21
2017-04-01    0.21
Freq: MS, dtype: float64, implied_vols_db=2017-01-01    0.45
2017-02-01    0.45
2017-03-01    0.45
2017-04-01    0.45
Freq: MS, dtype: float64)

In [74]:
res['price_model'].vol_model._terminal_vol(valuation_date, payoff.contract_dates, payoff.contract_dates + pd.DateOffset(-1))

(2017-03-01    0.00
 2017-03-02    0.45
 2017-03-03    0.45
 2017-03-04    0.45
 2017-03-05    0.45
 2017-03-06    0.45
 2017-03-07    0.45
 2017-03-08    0.45
 2017-03-09    0.45
 2017-03-10    0.45
 2017-03-11    0.45
 2017-03-12    0.45
 2017-03-13    0.45
 2017-03-14    0.45
 2017-03-15    0.45
 2017-03-16    0.45
 2017-03-17    0.45
 2017-03-18    0.45
 2017-03-19    0.45
 2017-03-20    0.45
 Freq: D, dtype: float64, {'theta': 2017-03-01             NaN
  2017-03-02    0.000000e+00
  2017-03-03    0.000000e+00
  2017-03-04    0.000000e+00
  2017-03-05    0.000000e+00
  2017-03-06    0.000000e+00
  2017-03-07    0.000000e+00
  2017-03-08    0.000000e+00
  2017-03-09    0.000000e+00
  2017-03-10    0.000000e+00
  2017-03-11    0.000000e+00
  2017-03-12    0.000000e+00
  2017-03-13    0.000000e+00
  2017-03-14    0.000000e+00
  2017-03-15    0.000000e+00
  2017-03-16    0.000000e+00
  2017-03-17    1.407986e-15
  2017-03-18    1.325164e-15
  2017-03-19    1.251543e-15
  2017-03-20   

In [78]:
res['spot_sims'].mean(axis=1)

simulation_dates
2017-03-01    25.500001
2017-03-02    25.500203
2017-03-03    25.500055
2017-03-04    25.500141
2017-03-05    25.499800
2017-03-06    25.499536
2017-03-07    25.499199
2017-03-08    25.499085
2017-03-09    25.498456
2017-03-10    25.498428
2017-03-11    25.498392
2017-03-12    25.498085
2017-03-13    25.499090
2017-03-14    25.499054
2017-03-15    25.498382
2017-03-16    25.498278
2017-03-17    25.498863
2017-03-18    25.499108
2017-03-19    25.498397
2017-03-20    25.498451
Freq: D, dtype: float64